<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/gemma_3n_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/google/gemma-3n-E4B-it

In [ ]:
# --- Force-install from source & debug paths ---
print("Starting fresh installation and path manipulation for Colab...")

# Install PyTorch first
print("Installing PyTorch...")
!pip install torch torchvision torchaudio

# Install transformers directly from the main branch
# We'll use --force-reinstall to be absolutely sure
print("Installing transformers from GitHub source (main branch) with force-reinstall...")
!pip install --no-deps --force-reinstall git+https://github.com/huggingface/transformers.git

# Install timm directly from the main branch (crucial for Gemma 3n's vision encoder)
print("Installing timm (PyTorch Image Models) from GitHub source (main branch)...")
!pip install --no-deps --force-reinstall git+https://github.com/huggingface/pytorch-image-models.git@main

# Install other necessary dependencies
# Note: We're installing dependencies separately because --no-deps was used above
print("Installing other core dependencies...")
!pip install -U accelerate bitsandbytes pillow requests pyyaml safetensors tokenizers tqdm regex packaging numpy filelock huggingface_hub

print("\n--- Installation complete. Attempting to force module loading paths... ---")

# DEBUG: Explicitly clear any existing module cache for transformers
import sys
if 'transformers' in sys.modules:
    del sys.modules['transformers']
if 'timm' in sys.modules:
    del sys.modules['timm']

# Get the path where pip installed the git package (usually in dist-packages)
import site
import os

transformers_site_path = None
for sp in site.getsitepackages():
    # Look for the .egg-info or .dist-info from the git install
    if os.path.exists(os.path.join(sp, 'transformers.egg-info')) or \
       os.path.exists(os.path.join(sp, 'transformers-*-py3.11.egg-info')): # More general
        transformers_site_path = sp
        break
    elif os.path.exists(os.path.join(sp, 'transformers')): # If it's a direct folder
         transformers_site_path = sp
         break

if transformers_site_path and transformers_site_path not in sys.path:
    # Add the site-packages path to the beginning of sys.path to prioritize it
    sys.path.insert(0, transformers_site_path)
    print(f"DEBUG: Added {transformers_site_path} to sys.path to prioritize imports.")
else:
    print("DEBUG: transformers site-packages path already in sys.path or not found correctly.")


# Now, explicitly try to import and check the version
from importlib.util import find_spec
spec = find_spec("transformers")
if spec:
    print(f"DEBUG: find_spec found transformers at: {spec.origin}")
else:
    print("DEBUG: find_spec could NOT locate transformers. This is critical.")


# DEBUG: Verify transformers installation and model type recognition after path manipulation
import transformers
print(f"DEBUG: Loaded transformers version: {transformers.__version__}")
print(f"DEBUG: Transformers path (after explicit path adjustments): {os.path.dirname(transformers.__file__)}")

try:
    from transformers.models.auto.configuration_auto import CONFIG_MAPPING
    print(f"DEBUG: 'gemma3n' in CONFIG_MAPPING: {'gemma3n' in CONFIG_MAPPING}")
except Exception as e:
    print(f"DEBUG: Failed to import CONFIG_MAPPING or check 'gemma3n' AFTER ALL EFFORTS: {e}")
    print("This indicates a severe, deep-rooted Colab environment issue.")

try:
    from transformers import Gemma3nConfig, Gemma3nForConditionalGeneration, Gemma3nProcessor
    print("DEBUG: Successfully imported Gemma3nConfig, Gemma3nForConditionalGeneration, Gemma3nProcessor directly.")
except ImportError as e:
    print(f"DEBUG: Failed to directly import Gemma3n classes AFTER ALL EFFORTS. Error: {e}")
    print("This is the absolute last resort for Colab.")


In [1]:
# Login to Hugging Face
from google.colab import userdata
from huggingface_hub import login

try:
    hf_token = userdata.get('HF_TOKEN')
    if hf_token is None:
        print("Warning: 'HF_TOKEN' not found in Colab Secrets. Please add it via the key icon on the left sidebar.")
        print("Falling back to interactive login...")
        login(add_to_git_credential=True)
    else:
        login(token=hf_token, add_to_git_credential=True)
    print("Hugging Face login successful.")
except Exception as e:
    print(f"Error accessing Colab secrets or logging into Hugging Face: {e}")
    print("Please ensure you have configured 'HF_TOKEN' in Colab secrets or manually provide token.")
    login(add_to_git_credential=True)

Hugging Face login successful.


In [ ]:
from transformers import pipeline
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3n-e4b-it",
    device="cuda",
    torch_dtype=torch.bfloat16,
)


In [ ]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=200)

In [4]:
print(output[0]["generated_text"][-1]["content"])

The animal on the candy is a **frog**. 🐸 

You can clearly see the silhouette of a frog on two of the candies. 


In [ ]:
from transformers import AutoProcessor, Gemma3nForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3n-e4b-it"

model = Gemma3nForConditionalGeneration.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16,).eval()

processor = AutoProcessor.from_pretrained(model_id)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg"},
            {"type": "text", "text": "Describe this image in detail."}
        ]
    }
]

inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)

In [2]:
print(decoded)

Captured from a slightly elevated perspective, the image showcases a vibrant pink cosmos flower in full bloom, with a small, fuzzy bumblebee diligently collecting nectar from its center. The flower's petals are a soft, delicate pink, radiating outwards from a bright yellow central disc. The petals have a slightly textured appearance, with subtle veins running along their length.

The bumblebee is positioned in the heart of the flower, its black and yellow stripes clearly visible against the yellow center. Its wings are slightly blurred
